In [1]:
!wget https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q3.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q4.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2020Q1.csv.zip

--2021-10-04 22:07:50--  https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
Resolving resources.lendingclub.com... 52.41.216.117, 52.11.25.247, 54.68.47.150
Connecting to resources.lendingclub.com|52.41.216.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: 'LoanStats_2019Q1.csv.zip.1'

LoanStats_2019Q1.cs     [   <=>              ]  20.27M  3.26MB/s    in 6.3s    

2021-10-04 22:07:57 (3.23 MB/s) - 'LoanStats_2019Q1.csv.zip.1' saved [21257309]

--2021-10-04 22:07:57--  https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
Resolving resources.lendingclub.com... 52.11.25.247, 54.68.47.150, 52.41.216.117
Connecting to resources.lendingclub.com|52.11.25.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: 'LoanStats_2019Q2.csv.zip.1'

LoanStats_2019Q2.cs     [              <=>   ]  23.10M  5.95MB/s    in 3.9s    

2021-10-04 22:08:01 (5.86 MB/s

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

In [11]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership", "annual_inc", 
    "verification_status", "pymnt_plan", "dti", "delinq_2yrs", 
    "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "total_acc", 
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt", 
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", 
    "total_rec_late_fee", "recoveries", "collection_recovery_fee", 
    "last_pymnt_amnt", "collections_12_mths_ex_med", "policy_code", 
    "application_type", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", 
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", 
    "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", 
    "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", 
    "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", 
    "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", 
    "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", 
    "mo_sin_rcnt_tl", "mort_acc", "mths_since_recent_bc", 
    "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", 
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
    "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", 
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", 
    "pub_rec_bankruptcies", "tax_liens", "tot_hi_cred_lim", 
    "total_bal_ex_mort", "total_bc_limit", "total_il_high_credit_limit", 
    "hardship_flag", "debt_settlement_flag",
    "loan_status"
]

target = "loan_status"

In [15]:
# Load the data
df1 = pd.read_csv(Path('/Users/melissa/Downloads/Supervised_Machine_Learning/Resources /Generator /LoanStats_2019Q1.csv.zip'), skiprows=1)[:-2]
df2 = pd.read_csv(Path('/Users/melissa/Downloads/Supervised_Machine_Learning/Resources /Generator /LoanStats_2019Q2.csv.zip'), skiprows=1)[:-2]
df3 = pd.read_csv(Path('/Users/melissa/Downloads/Supervised_Machine_Learning/Resources /Generator /LoanStats_2019Q3.csv.zip'), skiprows=1)[:-2]
df4 = pd.read_csv(Path('/Users/melissa/Downloads/Supervised_Machine_Learning/Resources /Generator /LoanStats_2019Q4.csv.zip'), skiprows=1)[:-2]

df = pd.concat([df1, df2, df3, df4]).loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)


low_risk_rows = df[df[target] == 'low_risk']
high_risk_rows = df[df[target] == 'high_risk']

#df = pd.concat([low_risk_rows, high_risk_rows.sample(n=len(low_risk_rows), replace=True)])
df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=42), high_risk_rows])
df = df.reset_index(drop=True)
df = df.rename({target:'target'}, axis="columns")
df

/Users/melissa/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,7000.0,0.1894,256.38,MORTGAGE,75000.0,Not Verified,n,28.62,0.0,2.0,...,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N,low_risk
1,40000.0,0.1614,975.71,MORTGAGE,102000.0,Source Verified,n,11.72,2.0,0.0,...,0.0,0.0,0.0,294664.0,109911.0,9000.0,71044.0,N,N,low_risk
2,11000.0,0.2055,294.81,RENT,45000.0,Verified,n,37.25,1.0,3.0,...,7.7,0.0,0.0,92228.0,36007.0,33000.0,46328.0,N,N,low_risk
3,4000.0,0.1612,140.87,MORTGAGE,38000.0,Not Verified,n,42.89,1.0,0.0,...,100.0,0.0,0.0,284273.0,52236.0,13500.0,52017.0,N,N,low_risk
4,14000.0,0.1797,505.93,MORTGAGE,43000.0,Source Verified,n,22.16,1.0,0.0,...,25.0,0.0,0.0,120280.0,88147.0,33300.0,78680.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,19975.0,0.2565,801.09,RENT,28000.0,Not Verified,n,28.42,0.0,0.0,...,16.7,0.0,0.0,50055.0,28192.0,18700.0,19055.0,N,N,high_risk
12176,15000.0,0.1774,540.34,RENT,50000.0,Verified,n,23.43,4.0,0.0,...,11.1,0.0,0.0,70324.0,57025.0,13300.0,54824.0,N,N,high_risk
12177,3600.0,0.1862,131.28,RENT,60000.0,Not Verified,n,28.80,0.0,1.0,...,0.0,0.0,0.0,83765.0,55156.0,14800.0,53065.0,N,N,high_risk
12178,15000.0,0.0881,475.68,MORTGAGE,62000.0,Source Verified,n,11.44,0.0,0.0,...,0.0,0.0,0.0,189930.0,23748.0,7000.0,32930.0,N,N,high_risk


In [16]:
df.to_csv('2019loans.csv', index=False)

In [17]:
# Load the data
validate_df = pd.read_csv(Path('/Users/melissa/Downloads/Supervised_Machine_Learning/Resources /Generator /LoanStats_2019Q1.csv.zip'), skiprows=1)[:-2]
validate_df = validate_df.loc[:, columns].copy()

# Drop the null columns where all values are null
validate_df = validate_df.dropna(axis='columns', how='all')

# Drop the null rows
validate_df = validate_df.dropna()

# Remove the `Issued` loan status
issued_mask = validate_df[target] != 'Issued'
validate_df = validate_df.loc[issued_mask]

# convert interest rate to numerical
validate_df['int_rate'] = validate_df['int_rate'].str.replace('%', '')
validate_df['int_rate'] = validate_df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = dict.fromkeys(['Current', 'Fully Paid'], 'low_risk') 
validate_df = validate_df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period', 'Charged Off'], 'high_risk')    
validate_df = validate_df.replace(x)

low_risk_rows = validate_df[validate_df[target] == 'low_risk']
high_risk_rows = validate_df[validate_df[target] == 'high_risk']

validate_df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=37), high_risk_rows])
validate_df = validate_df.reset_index(drop=True)
validate_df = validate_df.rename({target:'target'}, axis="columns")
validate_df

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/LoanStats_2020Q1.csv.zip'

In [8]:
validate_df.to_csv('2020Q1loans.csv', index=False)